https://stepik.org/lesson/1075715?unit=1085823

In [17]:
import sqlalchemy as db

In [18]:
db_name = 'db_files/task_4_(join).sqlite'

In [19]:
engine = db.create_engine(f'sqlite:///{db_name}')

In [20]:
conn = engine.connect()

In [39]:
query_drop_employee = "DROP TABLE IF EXISTS employee"
query_drop_tasks = "DROP TABLE IF EXISTS tasks"

In [63]:
conn.execute(db.text(query_drop_employee))
conn.commit()

In [42]:
conn.execute(db.text(query_drop_tasks))
conn.commit()

#### Создание таблиц

In [64]:
query_create_table_employee = """CREATE TABLE employee (
id INTEGER PRIMARY KEY,
name VARCHAR(128),
lastname  VARCHAR(128),
salary INTEGER DEFAULT 0,
manager_id INTEGER)"""

In [47]:
query_create_table_tasks = """CREATE TABLE tasks (
id INTEGER,
title VARCHAR(128),
employee_id INTEGER,
done_flg INTEGER)"""

In [65]:
conn.execute(db.text(query_create_table_employee))

In [49]:
conn.execute(db.text(query_create_table_tasks))

#### Заполнение таблиц

In [59]:
keys_employee = 'id', 'name', 'lastname', 'salary', 'manager_id'
data_employee = """| 1  | Thomas    | Nelson     | 150000 | NULL       |
| 2  | Willie    | Mitchell   | 90000  | 1          |
| 3  | David     | Garrett    | 170000 | 2          |
| 4  | Gary      | Richardson | 140000 | 1          |
| 5  | Donald    | Stewart    | 100000 | 3          |
| 6  | Frederick | White      | 70000  | 2          |
| 7  | Willie    | Gonzales   | 12000  | 1          |
| 8  | Angel     | Carter     | 990000 | 3          |"""

In [26]:
keys_tasks = 'id', 'title', 'employee_id', 'done_flg'
data_tasks = """| 1  | Task # 616 | 1           | 1        |
| 2  | Task # 741 | 4           | 1        |
| 3  | Task # 526 | 4           | 0        |
| 4  | Task # 588 | 1           | 1        |
| 5  | Task # 964 | 2           | 0        |
| 6  | Task # 282 | 4           | 0        |
| 7  | Task # 350 | 5           | 0        |
| 8  | Task # 523 | 7           | 1        |
| 9  | Task # 776 | 2           | 1        |
| 10 | Task # 155 | 2           | 1        |
| 11 | Task # 348 | 4           | 0        |
| 12 | Task # 703 | 3           | 1        |
| 13 | Task # 727 | 6           | 0        |
| 14 | Task # 131 | 5           | 0        |
| 15 | Task # 166 | 10          | 0        |
| 16 | Task # 167 | 10          | 1        |
| 17 | Task # 168 | 10          | 0        |"""

In [67]:
def set_table_to_default(connection, table_name, keys, data):
    default_data = [dict(zip(keys, map(str.strip, line.strip('|').split('|')))) 
                   for line in data.splitlines()]
    # print(*default_data, sep=chr(10))
    query_set = db.insert(db.Table(table_name, db.MetaData(), autoload_with=engine)).values(default_data)
    query_delete = db.delete(db.Table(table_name, db.MetaData()))
    connection.execute(query_delete)
    connection.execute(query_set)
    connection.commit()   

In [51]:
def reset_employee(connection=conn, table_name='employee', keys=keys_employee, data=data_employee):
    set_table_to_default(connection, table_name, keys, data)

In [52]:
def reset_tasks(connection=conn, table_name="tasks", keys=keys_tasks, data=data_tasks):
    set_table_to_default(connection, table_name, keys, data)

In [68]:
reset_employee()

In [69]:
reset_tasks()

В базе есть таблица employee, которая содержит данные о сотрудниках, а так же tasks с данными о задачах.
Структура таблиц следующая:

employee

id - идентификатор сотрудника
name - имя сотрудника
lastname - фамилия сотрудника
salary - зарплата сотрудника
manager_id - идентификатор менеджера сотрудника

tasks

id - идентификатор задачи
title - наименование задачи
employee_id - идентификатор сотрудника, который назначен исполнителем
done_flg - флаг выполнения задачи (1 - задача выполнена, 0 - задача не выполнена)



#### task 1
Сформируйте выборку, которая содержит в себе имя и фамилию сотрудника, а так же наименование задачи.

In [76]:
query_task_1 = """SELECT name, lastname, title 
FROM employee, tasks 
WHERE employee.id = tasks.employee_id"""

In [79]:
query_task_1_join = """SELECT name, lastname, title
FROM employee
JOIN tasks ON employee.id = tasks.employee_id"""

In [80]:
conn.execute(db.text(query_task_1_join)).fetchall()

[('Thomas', 'Nelson', 'Task # 616'),
 ('Gary', 'Richardson', 'Task # 741'),
 ('Gary', 'Richardson', 'Task # 526'),
 ('Thomas', 'Nelson', 'Task # 588'),
 ('Willie', 'Mitchell', 'Task # 964'),
 ('Gary', 'Richardson', 'Task # 282'),
 ('Donald', 'Stewart', 'Task # 350'),
 ('Willie', 'Gonzales', 'Task # 523'),
 ('Willie', 'Mitchell', 'Task # 776'),
 ('Willie', 'Mitchell', 'Task # 155'),
 ('Gary', 'Richardson', 'Task # 348'),
 ('David', 'Garrett', 'Task # 703'),
 ('Frederick', 'White', 'Task # 727'),
 ('Donald', 'Stewart', 'Task # 131')]

#### task_2 
Сформируйте выборку, которая содержит имена и фамилии сотрудников, у которых нет задач.

In [89]:
query_task_2_join = """SELECT name, lastname
FROM employee
LEFT OUTER JOIN tasks ON (employee.id = tasks.employee_id)
WHERE tasks.employee_id is NULL"""

In [85]:
query_task_2 = """SELECT name, lastname FROM employee
WHERE id NOT IN (SELECT employee_id FROM tasks)"""

In [86]:
conn.execute(db.text(query_task_2)).fetchall()

[('Angel', 'Carter')]

In [90]:
conn.execute(db.text(query_task_2_join)).fetchall()

[('Angel', 'Carter')]

#### task 3
Сформируйте выборку, которая содержит имена и фамилии сотрудников, у которых нет активных задач (либо задач вовсе нет, либо все задачи со значением 1 в поле done_flg).

In [107]:
query_task_3 = """SELECT name, lastname 
FROM employee
LEFT OUTER JOIN tasks ON employee.id = tasks.employee_id
WHERE tasks.employee_id is NULL or ALL(SELECT done_flg FROM tasks WHERE employee_id = employee.id"""

query_task_3 = """SELECT name, lastname 
FROM employee
WHERE id = ALL (SELECT employee_id FROM tasks WHERE done_flg = 1)"""



In [97]:
q31 = """SELECT employee_id, sum(done_flg) as done_tasks FROM tasks GROUP BY employee_id"""

In [108]:
conn.execute(db.text(query_task_3)).fetchall()

OperationalError: (sqlite3.OperationalError) near "ALL": syntax error
[SQL: SELECT name, lastname 
FROM employee
WHERE id = ALL (SELECT employee_id FROM tasks WHERE done_flg = 1)]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [98]:
conn.execute(db.text(q31)).fetchall()

[(1, 2), (2, 2), (3, 1), (4, 1), (5, 0), (6, 0), (7, 1), (10, 1)]